# Loader script for Neptune


In [87]:
# (c) 2022 Vanderbilt University, except Sparqler class: (c) Steven J. Baskauf (same license)
# This program is released under a GNU General Public License v3.0 http://www.gnu.org/licenses/gpl-3.0
# Author: Steve Baskauf
# Date: 2022-06-06

# ----------------
# Configuration
# ----------------
import requests
import json
from time import sleep
import datetime
import os
from os.path import exists
import pandas as pd
import yaml

# AWS Python SDK
import boto3
# import botocore  # not used

import csv
from pathlib import Path
import sys
import re # regex
import urllib.parse

# Global variables
loader_endpoint_url = 'https://triplestore1.cluster-cml0hq81gymg.us-east-1.neptune.amazonaws.com:8182/sparql'
reader_endpoint_url = 'https://5j6diw4i0h.execute-api.us-east-1.amazonaws.com/sparql'
local_upload_directory = '/Users/baskausj/triplestore_upload/'
s3_bucket_name = 'triplestore-upload'
utc_offset = '-06:00'

# See https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3.html#uploads
# local_file_path = local_directory + local_filename
# s3_file_key = local_filename
# s3_file_key = s3_iiif_project_directory + '/' + subdirectory + '/' + local_filename

def parse_filename(filename):
    """Separates and returns the root of a filename and the extension as a tuple."""
    pieces = filename.split('.')
    file_name_root = '.'.join(pieces[:-1])
    extension = pieces[len(pieces)-1]
    return file_name_root, extension

def csv_read(path, **kwargs):
    """Loads a CSV table into a Pandas DataFrame with all cells as strings and blank cells as empty strings
    
    Keyword argument:
    rows -- the number of rows of the table to return when used for testing. When omitted, all rows are returned.
    """
    dataframe = pd.read_csv(path, na_filter=False, dtype = str)
    if 'rows' in kwargs:
        return dataframe.head(kwargs['rows']).copy(deep=True)
    else:
        return dataframe

class Sparqler:
    """Build SPARQL queries of various sorts

    Parameters
    -----------
    useragent : str
        Required if using the Wikidata Query Service, otherwise optional.
        Use the form: appname/v.v (URL; mailto:email@domain.com)
        See https://meta.wikimedia.org/wiki/User-Agent_policy
    endpoint: URL
        Defaults to Wikidata Query Service if not provided.
    method: str
        Possible values are "post" (default) or "get". Use "get" if read-only query endpoint.
        Must be "post" for update endpoint.
    sleep: float
        Number of seconds to wait between queries. Defaults to 0.1
        
    Required modules:
    -------------
    import requests
    from time import sleep
    """
    def __init__(self, **kwargs):
        # attributes for all methods
        try:
            self.http_method = kwargs['method']
        except:
            self.http_method = 'post' # default to POST
        try:
            self.endpoint = kwargs['endpoint']
        except:
            self.endpoint = 'https://query.wikidata.org/sparql' # default to Wikidata endpoint
        try:
            self.useragent = kwargs['useragent']
        except:
            if self.endpoint == 'https://query.wikidata.org/sparql':
                print('You must provide a value for the useragent argument when using the Wikidata Query Service.')
                print()
                raise KeyboardInterrupt # Use keyboard interrupt instead of sys.exit() because it works in Jupyter notebooks
            else:
                self.useragent = ''
        try:
            self.sleep = kwargs['sleep']
        except:
            self.sleep = 0.1 # default throtting of 0.1 seconds

        self.requestheader = {}
        if self.useragent:
            self.requestheader['User-Agent'] = self.useragent
        
        if self.http_method == 'post':
            self.requestheader['Content-Type'] = 'application/x-www-form-urlencoded'

    def query(self, query_string, **kwargs):
        """Sends a SPARQL query to the endpoint.
        
        Parameters
        ----------
        form : str
            The SPARQL query form.
            Possible values are: "select" (default), "ask", "construct", and "describe".
        mediatype: str
            The response media type (MIME type) of the query results.
            Some possible values for "select" and "ask" are: "application/sparql-results+json" (default) and "application/sparql-results+xml".
            Some possible values for "construct" and "describe" are: "text/turtle" (default) and "application/rdf+xml".
        verbose: bool
            Prints status when True. Defaults to False.
        default: list of str
            The graphs to be merged to form the default graph. List items must be URIs in string form.
            If omitted, no graphs will be specified and default graph composition will be controlled by FROM clauses
            in the query itself. 
            See https://www.w3.org/TR/sparql11-query/#namedGraphs and https://www.w3.org/TR/sparql11-protocol/#dataset
            for details.
        named: list of str
            Graphs that may be specified by IRI in a query. List items must be URIs in string form.
            If omitted, named graphs will be specified by FROM NAMED clauses in the query itself.
            
        Returns
        -------
        If the form is "select" and mediatype is "application/json", a list of dictionaries containing the data.
        If the form is "ask" and mediatype is "application/json", a boolean is returned.
        If the mediatype is "application/json" and an error occurs, None is returned.
        For other forms and mediatypes, the raw output is returned.

        Notes
        -----
        To get UTF-8 text in the SPARQL queries to work properly, send URL-encoded text rather than raw text.
        That is done automatically by the requests module for GET. I guess it also does it for POST when the
        data are sent as a dict with the urlencoded header. 
        See SPARQL 1.1 protocol notes at https://www.w3.org/TR/sparql11-protocol/#query-operation        
        """
        try:
            query_form = kwargs['form']
        except:
            query_form = 'select' # default to SELECT query form
        try:
            media_type = kwargs['mediatype']
        except:
            #if query_form == 'construct' or query_form == 'describe':
            if query_form == 'construct':
                media_type = 'text/turtle'
            else:
                media_type = 'application/sparql-results+json' # default for SELECT and ASK query forms
        self.requestheader['Accept'] = media_type
        try:
            verbose = kwargs['verbose']
        except:
            verbose = False # default to no printouts
            
        # Build the payload dictionary (query and graph data) to be sent to the endpoint
        payload = {'query' : query_string}
        try:
            payload['default-graph-uri'] = kwargs['default']
        except:
            pass
        
        try:
            payload['named-graph-uri'] = kwargs['named']
        except:
            pass

        if verbose:
            print('querying SPARQL endpoint')

        start_time = datetime.datetime.now()
        if self.http_method == 'post':
            response = requests.post(self.endpoint, data=payload, headers=self.requestheader)
        else:
            response = requests.get(self.endpoint, params=payload, headers=self.requestheader)
        elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
        self.response = response.text
        sleep(self.sleep) # Throttle as a courtesy to avoid hitting the endpoint too fast.

        if verbose:
            print('done retrieving data in', int(elapsed_time), 's')

        if query_form == 'construct' or query_form == 'describe':
            return response.text
        else:
            if media_type != 'application/sparql-results+json':
                return response.text
            else:
                try:
                    data = response.json()
                except:
                    return None # Returns no value if an error. 

                if query_form == 'select':
                    # Extract the values from the response JSON
                    results = data['results']['bindings']
                else:
                    results = data['boolean'] # True or False result from ASK query 
                return results           

    def update(self, request_string, **kwargs):
        """Sends a SPARQL update to the endpoint.
        
        Parameters
        ----------
        mediatype : str
            The response media type (MIME type) from the endpoint after the update.
            Default is "application/json"; probably no need to use anything different.
        verbose: bool
            Prints status when True. Defaults to False.
        default: list of str
            The graphs to be merged to form the default graph. List items must be URIs in string form.
            If omitted, no graphs will be specified and default graph composition will be controlled by USING
            clauses in the query itself. 
            See https://www.w3.org/TR/sparql11-update/#deleteInsert
            and https://www.w3.org/TR/sparql11-protocol/#update-operation for details.
        named: list of str
            Graphs that may be specified by IRI in the graph pattern. List items must be URIs in string form.
            If omitted, named graphs will be specified by USING NAMED clauses in the query itself.
        """
        try:
            media_type = kwargs['mediatype']
        except:
            media_type = 'application/json' # default response type after update
        self.requestheader['Accept'] = media_type
        try:
            verbose = kwargs['verbose']
        except:
            verbose = False # default to no printouts
        
        # Build the payload dictionary (update request and graph data) to be sent to the endpoint
        payload = {'update' : request_string}
        try:
            payload['using-graph-uri'] = kwargs['default']
        except:
            pass
        
        try:
            payload['using-named-graph-uri'] = kwargs['named']
        except:
            pass

        if verbose:
            print('beginning update')
            
        start_time = datetime.datetime.now()
        response = requests.post(self.endpoint, data=payload, headers=self.requestheader)
        elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
        self.response = response.text
        sleep(self.sleep) # Throttle as a courtesy to avoid hitting the endpoint too fast.

        if verbose:
            print('done updating data in', int(elapsed_time), 's')

        if media_type != 'application/json':
            return response.text
        else:
            try:
                data = response.json()
            except:
                return None # Returns no value if an error converting to JSON (e.g. plain text) 
            return data           

    def load(self, file_location, graph_uri, **kwargs):
        """Loads an RDF document into a specified graph.
        
        Parameters
        ----------
        s3 : str
            Name of an AWS S3 bucket containing the file. Omit load a generic URL.
        verbose: bool
            Prints status when True. Defaults to False.
        
        Notes
        -----
        The triplestore may or may not rely on receiving a correct Content-Type header with the file to
        determine the type of serialization. Blazegraph requires it, AWS Neptune does not and apparently
        interprets serialization based on the file extension.
        """
        try:
            s3 = kwargs['s3']
        except:
            s3 = ''
        try:
            verbose = kwargs['verbose']
        except:
            verbose = False # default to no printouts

        if s3:
            request_string = 'LOAD <https://' + s3 + '.s3.amazonaws.com/' + file_location + '> INTO GRAPH <' + graph_uri + '>'
        else:
            request_string = 'LOAD <' + file_location + '> INTO GRAPH <' + graph_uri + '>'
        
        if verbose:
            print('Loading file:', file_location, ' into graph: ', graph_uri)
        data = self.update(request_string, verbose=verbose)
        return data

    def drop(self, graph_uri, **kwargs):
        """Drop a specified graph.
        
        Parameters
        ----------
        verbose: bool
            Prints status when True. Defaults to False.
        """
        try:
            verbose = kwargs['verbose']
        except:
            verbose = False # default to no printouts

        request_string = 'DROP GRAPH <' + graph_uri + '>'

        if verbose:
            print('Deleting graph:', graph_uri)
        data = self.update(request_string, verbose=verbose)
        return data

# -----------------
# Begin main script
# -----------------
sve = Sparqler(endpoint=reader_endpoint_url, method='get', sleep=0)
neptune = Sparqler(endpoint=loader_endpoint_url, sleep=0)

## List graphs loaded in Neptune

In [86]:
query_string = '''select distinct ?graph where {
graph ?graph {?s ?o ?p.}
}
order by ?graph'''

data = sve.query(query_string)
#print(json.dumps(data, indent=2))

for graph in data:
    print(graph['graph']['value'])

http://AATOut_1Subjects
http://AATOut_2Terms
http://AATOut_AssociativeRels
http://AATOut_ContribRels
http://AATOut_Contribs
http://AATOut_HierarchicalRels
http://AATOut_LCSHAlignment
http://AATOut_Lang_sameAs
http://AATOut_Notations
http://AATOut_ObsoleteSubjects
http://AATOut_OrderedCollections
http://AATOut_ScopeNotes
http://AATOut_SemanticLinks
http://AATOut_SourceRels
http://AATOut_Sources
http://AATOut_WikidataCoref
http://nomenclature_2022-02-02
http://syriaca.org/bibl#graph
https://sparql.vanderbilt.edu/graphs


## Use AWS SDK to upload files to the public S3 bucket

In [79]:
# ----------------
# Upload RDF triples to S3 bucket
# ----------------

s3 = boto3.client('s3')

# The upload directory should contain only RDF serializations that are to be uploaded.
file_list = os.listdir(local_upload_directory)
try:
    file_list.remove('.DS_Store') # Macs sometimes generate this hidden file - try to remove from upload list
except:
    pass

for file_name in file_list:
    #file_name_root = parse_filename(file_name)[0]
    local_file_path = local_upload_directory + file_name
    s3_file_key = file_name

    print('Uploading to s3:', file_name)
    start_time = datetime.datetime.now()
    s3.upload_file(local_file_path, s3_bucket_name, s3_file_key)
    elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
    print('Upload complete in', elapsed_time, 's.')


Uploading to s3: bibl-pelagios.rdf
Upload complete in 6.516015 s.


## Connect and test the SSH tunnel

In [ ]:
# Don't run this cell until we figure out how to make this command complete. The cell never stops executing and
# that locks up the rest of the notebook. For now, run it in a separate terminal window (left open).

# The EC2 server acting as the SSH tunnel has the IP address: 35.173.230.91

# The following lines need to be added to the ~/.ssh/config file:

"""
host neptune
 ForwardAgent yes
 User ec2-user
 HostName 35.173.230.91
 IdentitiesOnly yes
 IdentityFile ~/NeptuneSSHtunnel.pem
 LocalForward 8182 triplestore1.cluster-cml0hq81gymg.us-east-1.neptune.amazonaws.com:8182
"""

# and the NeptuneSSHtunnel.pem file needs to be in the home directory

# Start up SSH tunnel
#os.system('ssh neptune -N') # uncomment if we get it to work

In [80]:
# Run this to make sure SSH tunnel is working
try:
    response = requests.get(loader_endpoint_url + '/status')
    print(response.json())
except Exception as e:
    print('error', e.args[0])
  

{'acceptedQueryCount': 56, 'runningQueryCount': 0, 'queries': []}


## Load files in S3 into triplestore

For reference, Neptune's conformance with SPARQL is described at https://docs.aws.amazon.com/neptune/latest/userguide/feature-sparql-compliance.html . Most specifically, triples are always associated with a graph, even if one is not explicitly specified. See https://docs.aws.amazon.com/neptune/latest/userguide/feature-sparql-compliance.html#sparql-default-graph . The fallback URI `http://aws.amazon.com/neptune/vocab/v01/DefaultNamedGraph` is used.

Setup and load data

In [81]:
# Instantiate Neptune SPARQL object
neptune = Sparqler(endpoint=loader_endpoint_url, sleep=0)

# Load the query prefix section from a text file
with open('prefixes.txt', 'rt', encoding='utf-8') as file_object:
    prefixes_text = file_object.read()
    
# Load the named graph data CSV configuration
if exists('named_graphs_config.yml'):
    with open('named_graphs_config.yml') as file_object:
        config_data = yaml.safe_load(file_object)
else:
    print('Must have a named_graphs_config.yml file for this script to operate.')
    print()
    raise KeyboardInterrupt

# Load the data about named graphs to be updated
named_graphs_df = csv_read('named_graphs.csv')

# Load the data relating the graph names to the datafiles that contain the serializations
graph_file_associations_df = csv_read('graph_file_associations.csv')

# Check that all of the files exist before moving on
for index, file in graph_file_associations_df.iterrows():
    if not exists(local_upload_directory + file['filename']):
        print(file['filename'], 'does not exist in the directory:', local_upload_directory)
        print('Put it there, or adjust the graph_file_associations.csv file, and try again.')
        print()
        raise KeyboardInterrupt

#    else:
#        print(file['filename'], 'exists')


Load the triples from the S3 bucket into the triplestore

In [82]:
for index, file in graph_file_associations_df.iterrows():
    graph_name = file['sd:name']
    data = neptune.load(file['filename'], graph_name, s3=s3_bucket_name, verbose=True)
    print(json.dumps(data, indent=2))


Loading file: bibl-pelagios.rdf  into graph:  http://syriaca.org/bibl#graph
beginning update
done updating data in 7 s
[
  {
    "type": "UpdateEvent",
    "totalElapsedMillis": 2288,
    "elapsedMillis": 2287,
    "connFlush": 0,
    "batchResolve": 0,
    "whereClause": 0,
    "deleteClause": 0,
    "insertClause": 0
  },
  {
    "type": "Commit",
    "totalElapsedMillis": 7243
  }
]


## Update the metadata about graphs to reflect the changes

If there are old triples in the graphs metadata associated with the named graphs being updated, they are deleted from the <https://sparql.vanderbilt.edu/graphs> graph. See https://www.w3.org/TR/sparql11-update/#deleteWhere for reference

In [83]:
# Note that attempting to delete triples that don't exist or from a graph that doesn't exist has no effect
# and generates no errors. 

"""Generates a request string of the form:
delete where {
graph <https://sparql.vanderbilt.edu/graphs> {
<http://syriaca.org/bibl#graph> ?o ?p.

...

}}
"""

# NOTE: prefixes not needed since the subject IRI is unabbreviated.
request_string = '''delete where {
graph <https://sparql.vanderbilt.edu/graphs> {\n'''
for index, graph in named_graphs_df.iterrows():
    request_string += '<' + graph['sd:name'] + '> ?o ?p.\n'
request_string += '}}\n'    
print(request_string)
        
data = neptune.update(request_string, verbose=True)
print(json.dumps(data, indent=2))


delete where {
graph <https://sparql.vanderbilt.edu/graphs> {
<http://syriaca.org/bibl#graph> ?o ?p.
}}

beginning update
done updating data in 0 s
[
  {
    "type": "UpdateEvent",
    "totalElapsedMillis": 9,
    "elapsedMillis": 8,
    "connFlush": 0,
    "batchResolve": 0,
    "whereClause": 7,
    "deleteClause": 0,
    "insertClause": 0
  },
  {
    "type": "Commit",
    "totalElapsedMillis": 10
  }
]


Generate the new metadata for the service description and load it into the <https://sparql.vanderbilt.edu/graphs> graph. See https://www.w3.org/TR/sparql11-update/#insertData for reference. Note: unlike the examples at https://www.w3.org/TR/sparql11-service-description/#example-turtle and https://www.w3.org/TR/void/#sparql-sd , we don't use a blank node for the sd:NamedGraph instance. This makes it easier to manage linking it to the graph collection.

In [84]:
"""Generates a request string of the form:

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix sd: <http://www.w3.org/ns/sparql-service-description#>
prefix dc: <http://purl.org/dc/elements/1.1/>
prefix dcterms: <http://purl.org/dc/terms/>
prefix tdwgutility: <http://rs.tdwg.org/dwc/terms/attributes/>
insert data {
graph <https://sparql.vanderbilt.edu/graphs> {
<http://syriaca.org/bibl#graph> dcterms:modified "2022-06-06T14:40:05.430286-06:00"^^xsd:dateTime.
<http://syriaca.org/bibl#graph> sd:name <http://syriaca.org/bibl#graph>.
<http://syriaca.org/bibl#graph> sd:graph <http://syriaca.org/bibl>.
<http://syriaca.org/bibl#graph> dcterms:issued "2018-03-06"^^xsd:date.
<http://syriaca.org/bibl#graph> dc:publisher "syriaca.org".
<http://syriaca.org/bibl#graph> rdf:type sd:NamedGraph.
<http://syriaca.org/bibl#graph> dcterms:isPartOf <http://syriaca.org/>.
<http://syriaca.org/bibl#graph> tdwgutility:status "production".

...

}}
"""
dcterms_modified = datetime.datetime.now().isoformat() + utc_offset

request_string = prefixes_text + '''insert data {
graph <https://sparql.vanderbilt.edu/graphs> {\n'''
for index, graph in named_graphs_df.iterrows():
    named_graph_iri = graph['sd:name']
    request_string += '<' + named_graph_iri + '> dcterms:modified "' + dcterms_modified + '"^^xsd:dateTime.\n'
    for column in config_data:
        triple_pattern = '<' + named_graph_iri + '> ' + column['column_header'] + ' '
        if column['object_type'] == 'iri':
            triple_pattern += '<' + graph[column['column_header']] + '>'
        elif column['object_type'] == 'curie':
            triple_pattern += graph[column['column_header']]
        elif column['object_type'] == 'literal':
            triple_pattern += '"' + graph[column['column_header']] + '"'
            if 'datatype' in column:
                triple_pattern += '^^' + column['datatype']
            if 'lang' in column:
                triple_pattern += '@' + column['lang']
        triple_pattern += '.\n'
        request_string += triple_pattern
request_string += '}}\n'    
print(request_string)
        
data = neptune.update(request_string, verbose=True)
print(json.dumps(data, indent=2))


prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix sd: <http://www.w3.org/ns/sparql-service-description#>
prefix void: <http://rdfs.org/ns/void#>
prefix dc: <http://purl.org/dc/elements/1.1/>
prefix dcterms: <http://purl.org/dc/terms/>
prefix tdwgutility: <http://rs.tdwg.org/dwc/terms/attributes/>
insert data {
graph <https://sparql.vanderbilt.edu/graphs> {
<http://syriaca.org/bibl#graph> dcterms:modified "2022-06-06T18:04:54.026168-06:00"^^xsd:dateTime.
<http://syriaca.org/bibl#graph> sd:name <http://syriaca.org/bibl#graph>.
<http://syriaca.org/bibl#graph> sd:graph <http://syriaca.org/bibl>.
<http://syriaca.org/bibl#graph> dcterms:issued "2018-03-06"^^xsd:date.
<http://syriaca.org/bibl#graph> dc:publisher "syriaca.org".
<http://syriaca.org/bibl#graph> rdf:type sd:NamedGraph.
<http://syriaca.org/bibl#graph> dcterms:isPartOf <http://syriaca.org/>.
<http://syriaca.org/bibl#g

Need to add the named graph to the GraphCollection if it isn't there already and update the modified dateTime for the https://sparql.vanderbilt.edu/graphs graph. This uses the WITH clause to carry out both the insert and delete sequentially. See https://www.w3.org/TR/sparql11-update/#deleteInsert for reference.

In [85]:
# Update the modified time for the graph of graphs data 
dcterms_modified = datetime.datetime.now().isoformat() + utc_offset
request_string = '''prefix dcterms: <http://purl.org/dc/terms/>

with <https://sparql.vanderbilt.edu/graphs> 
delete {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified ?dateTime.
}
insert {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified "''' + dcterms_modified + '''"^^xsd:dateTime.
}
where {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified ?dateTime.
}'''

print(request_string)

data = neptune.update(request_string, verbose=True)
print(json.dumps(data, indent=2))
print()

# Delete the links from the GraphCollection to the named graphs so that the triples won't get duplicated when added
# NOTE: this may not be necessary since it doesn't appear that duplicates get generated if we don't delete. 
# But it take negligible time and doesn't hurt anything.
request_string = '''prefix sd: <http://www.w3.org/ns/sparql-service-description#>
delete where {
graph <https://sparql.vanderbilt.edu/graphs> {\n'''
for index, graph in named_graphs_df.iterrows():
    request_string += '<https://sparql.vanderbilt.edu/graphs#collection> sd:namedGraph <' + graph['sd:name'] + '>.\n'
request_string += '}}\n'    
print(request_string)
        
data = neptune.update(request_string, verbose=True)
print(json.dumps(data, indent=2))
print()

# Now insert the linking triples
request_string = '''prefix sd: <http://www.w3.org/ns/sparql-service-description#>
insert data {
graph <https://sparql.vanderbilt.edu/graphs> {\n'''
for index, graph in named_graphs_df.iterrows():
    request_string += '<https://sparql.vanderbilt.edu/graphs#collection> sd:namedGraph <' + graph['sd:name'] + '>.\n'
request_string += '}}\n'    
print(request_string)
        
data = neptune.update(request_string, verbose=True)
print(json.dumps(data, indent=2))
print()


prefix dcterms: <http://purl.org/dc/terms/>

with <https://sparql.vanderbilt.edu/graphs> 
delete {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified ?dateTime.
}
insert {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified "2022-06-06T18:05:02.249377-06:00"^^xsd:dateTime.
}
where {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified ?dateTime.
}
beginning update
done updating data in 0 s
[
  {
    "type": "UpdateEvent",
    "totalElapsedMillis": 23,
    "elapsedMillis": 21,
    "connFlush": 0,
    "batchResolve": 0,
    "whereClause": 18,
    "deleteClause": 1,
    "insertClause": 0
  },
  {
    "type": "Commit",
    "totalElapsedMillis": 26
  }
]

prefix sd: <http://www.w3.org/ns/sparql-service-description#>
delete where {
graph <https://sparql.vanderbilt.edu/graphs> {
<https://sparql.vanderbilt.edu/graphs#collection> sd:namedGraph <http://syriaca.org/bibl#graph>.
}}

beginning update
done updating data in 0 s
[
  {
    "type": "UpdateEvent",
    "totalElapsedMillis": 7,

In [78]:
# One-time generation of service description metadata

dcterms_modified = datetime.datetime.now().isoformat() + utc_offset

request_string = '''prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix dc: <http://purl.org/dc/elements/1.1/>
prefix dcterms: <http://purl.org/dc/terms/>
prefix sd: <http://www.w3.org/ns/sparql-service-description#>
prefix tdwgutility: <http://rs.tdwg.org/dwc/terms/attributes/>

insert data {
graph <https://sparql.vanderbilt.edu/graphs> {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified "''' + dcterms_modified + '''"^^xsd:dateTime;
    sd:name <https://sparql.vanderbilt.edu/graphs>;
    dc:publisher "Vanderbilt Heard Libraries";
    a sd:NamedGraph;
    tdwgutility:status "production".
<https://sparql.vanderbilt.edu/sparql#service> a sd:Service;
    sd:endpoint <https://sparql.vanderbilt.edu/sparql>;
    sd:availableGraphs <https://sparql.vanderbilt.edu/graphs#collection>.
<https://sparql.vanderbilt.edu/graphs#collection> a sd:GraphCollection;
    sd:namedGraph <https://sparql.vanderbilt.edu/graphs>.
}}
'''

print(request_string)

data = neptune.update(request_string, verbose=True)
print(json.dumps(data, indent=2))


prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix dc: <http://purl.org/dc/elements/1.1/>
prefix dcterms: <http://purl.org/dc/terms/>
prefix sd: <http://www.w3.org/ns/sparql-service-description#>
prefix tdwgutility: <http://rs.tdwg.org/dwc/terms/attributes/>

insert data {
graph <https://sparql.vanderbilt.edu/graphs> {
<https://sparql.vanderbilt.edu/graphs> dcterms:modified "2022-06-06T18:02:45.642038-06:00"^^xsd:dateTime;
    sd:name <https://sparql.vanderbilt.edu/graphs>;
    dc:publisher "Vanderbilt Heard Libraries";
    a sd:NamedGraph;
    tdwgutility:status "production".
<https://sparql.vanderbilt.edu/sparql#service> a sd:Service;
    sd:endpoint <https://sparql.vanderbilt.edu/sparql>;
    sd:availableGraphs <https://sparql.vanderbilt.edu/graphs#collection>.
<https://sparql.vanderbilt.edu/graphs#collection> a sd:GraphCollection;
    sd:namedGraph <https://sparql.vanderbilt.edu/graphs>.
}}

beginning update
done updatin

## Code for dropping graphs

(under development)

In [7]:
# POST SPARQL Update DROP GRAPH command

query_string = 'DROP GRAPH <' + graph_iri + '>'
start_time = datetime.datetime.now()
response = requests.post(loader_endpoint_url + '/sparql', data=query_string.encode('utf-8'), headers=update_request_header_dictionary)
#print(response.text) # uncomment to view the raw response, e.g. if you are getting an error
data = response.json()
print(json.dumps(data, indent = 2))

elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
print('time to delete:', int(elapsed_time), 's')

[
  {
    "type": "UpdateEvent",
    "totalElapsedMillis": 7,
    "elapsedMillis": 2,
    "connFlush": 0,
    "batchResolve": 0,
    "whereClause": 0,
    "deleteClause": 0,
    "insertClause": 0
  },
  {
    "type": "Commit",
    "totalElapsedMillis": 11
  }
]
time to delete: 0 s


## DO NOT RUN this cell! It will delete everything in the triplestore

Actually, I've left it with an error so that it will fail if it's accidentally run.

In [ ]:
# POST SPARQL Update DROP ALL command
# Warning! This deletes all triples for all graphs !!!!

query_string = 'DROP ALL'
start_time = datetime.datetime.now()
response = requests.post(loader_endpoint_url + '/sparql', data=query_string.encode('utf-8'), headers=update_request_header_dictionary)
#print(response.text) # uncomment to view the raw response, e.g. if you are getting an error
data = response.json()
print(json.dumps(data, indent = 2))

elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
print('time to load:', int(elapsed_time), 's')

# Obsolete development code

Left for historical reference

## Code to test the Neptune-specific loader

There did not seem to be any real benefit to using this loader with respect to speed. It also does not have any mechanism for specifying the graph into which triples should be loaded, so the only way to load them into a specific graph is to use the n-quads format. See following cell to convert from n-triples to n-quads.

In [ ]:
# POST loader command

loader_request_header_dictionary = {
        'Accept' : 'application/json',
        'Content-Type': 'application/json'
    }

rdf_format = 'nquads'
#rdf_format = 'ntriples'

data = '''
    {
      "source" : "s3://'''+ s3_bucket_name + '/' + s3_file_key + '''",
      "format" : "'''  + rdf_format + '''",
      "iamRoleArn" : "arn:aws:iam::555751041262:role/neptuneloadfroms3",
      "region" : "us-east-1",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "updateSingleCardinalityProperties" : "FALSE",
      "queueRequest" : "TRUE"
    }'''

start_time = datetime.datetime.now()

# Send request to load
response = requests.post(loader_endpoint_url + '/loader', data=data.encode('utf-8'), headers=loader_request_header_dictionary)
data = response.json()
print(json.dumps(data, indent = 2))
load_id = data['payload']['loadId']

# Check status of load once per second
completed = False
while not completed:
    response = requests.get(loader_endpoint_url + '/loader/' + load_id)
    data = response.json()
    #print(json.dumps(data, indent = 2))
    print(data['payload']['overallStatus']['status'])
    if data['payload']['overallStatus']['status'] == 'LOAD_COMPLETED' or data['payload']['overallStatus']['status'] == 'LOAD_FAILED':
        completed = True
    sleep(1)
elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
print('time to load:', int(elapsed_time), 's')

## Converting n-triples to n-quads to specify graph


In [ ]:
file_list = os.listdir(local_directory)
file_list.remove('.DS_Store')

# Note: assumes all files are n-triples serialization with .nt file extensions.
for file_name in file_list:
    file_name_root = parse_filename(file_name)[0]
    print('converting:', file_name)
    output_filename = file_name_root + '.nq'
    graph_string = ' <http://' + file_name_root + '> .'

    output_file_object = open(local_directory + output_filename, 'wt', encoding='utf-8')
    input_file_object = open(local_directory + file_name, 'rt', encoding='utf-8')

    for line in input_file_object:
        line_text = line.strip() # remove trailing newline
        line_text = line_text[:-1] + graph_string # remove period at end.
        print(line_text, file=output_file_object)

    output_file_object.close()
    input_file_object.close()

print('done')